In [1]:
man = [1,0,0,0,1,1,0,0,0]
man

[1, 0, 0, 0, 1, 1, 0, 0, 0]

In [547]:
### WIDTH = 10;

FP16_MAN_LEN = 10
FP16_EXP_LEN = 5
FP16_EXP_BIAS = 2**(FP16_EXP_LEN-1)-1         # 15
FP32_MAN_LEN = 23
FP32_EXP_LEN = 8
FP32_EXP_BIAS = 2**(FP32_EXP_LEN-1)-1         # 127
FP64_MAN_LEN = 52
FP64_EXP_LEN = 11
FP64_EXP_BIAS = 2**(FP64_EXP_LEN-1)-1         # 1023


FMT_FP16 = [1,FP16_MAN_LEN,FP16_EXP_BIAS,FP16_EXP_LEN]
FMT_FP32 = [1,FP32_MAN_LEN,FP32_EXP_BIAS,FP32_EXP_LEN]

# CAUTION: Python List and Binary indexes left-to-right
# Call list reverse "a.reverse()"" before indexing in verilog fashion

# Useful Lambda Functions

# Convert Binary String (without '0b') to Binary Array
strdnum  = lambda s    : list(int(x) for x in s)
# Convert Binary Array to Binary String (without '0b')
numdstr  = lambda n    : "".join(str(x) for x in n)
# Convert Binary Array to Integer
bin2int  = lambda b    : int(numdstr(b),2)
# Convert Integer to Binary Array
int2bin  = lambda i, w : strdnum(bin(i)[2:].zfill(w))
# Reverse Indexing
indexrev = lambda a, i : a[len(a)-1-i]
# Check If an Array Only Has One Variety of Element
is_all   = lambda a, e : (a.count(e)==len(a))

# Convert binary array to Python float value
def fpdecode(fmt,pack):
    exp_len      = fmt[3]
    exp_bias     = fmt[2]
    man_len      = fmt[1]
    has_denorm   = fmt[0]==1
    force_denorm = fmt[0]==3
    
    sgn = pack[0]
    exp = pack[1:exp_len+1]
    man = pack[exp_len+1:]
    
    if (is_all(exp,0) and has_denorm) or force_denorm:
        # DENORM
        t = float(bin2int(man))*2**(-man_len+1)
        t = t*(2**(-exp_bias))
    
    # SPECIAL numbers are not dealt-with in this implementation
    elif (is_all(exp,1)) :
        print("ERROR: SPECIAL NUMBER");
        return float("nan")
    else:
        # NORM
        t = float(bin2int([1] + man))*2**(-man_len)
        t = t*(2**(float(bin2int(exp))-exp_bias))
        
    if sgn == 1:
            t = t*-1
    return t

# Conver float value to binary array
import struct
from math import log
def double2binary(num):
    return ''.join('{:0>8b}'.format(c) for c in struct.pack('!d', num))

def half2binary(num):
    return ''.join('{:0>8b}'.format(c) for c in struct.pack('!e', num))

# Convert Python float (double) value to binary array in fmt
# FP16 doesn't need this function:
#    just call half2binary to get a binary string, 
#    and then convert it to a binary array with strdnum(s)
def fpencode(fmt,num):
    exp_len      = fmt[3]
    exp_bias     = fmt[2]
    man_len      = fmt[1]
    has_denorm   = fmt[0]==1
    force_denorm = fmt[0]==3
    
    a = double2binary(num);
    sgn = a[0]
    exp64 = a[1:FP64_EXP_LEN+1]
    man64 = a[FP64_EXP_LEN+1:]
    
    # Convert Exponent
    exp64i = bin2int(exp64)
    exp = exp64i - FP64_EXP_BIAS + exp_bias
    if is_all(exp64,'0'):
        # Zeros
        return [int(sgn)] + [0]*exp_len + [0]*man_len
    if is_all(exp64,'1'):
        # Infinity
        return [int(sgn)] + [1]*exp_len + [0]*man_len
    if abs(exp) >= 2**(exp_len)-1:
        # Overflow, not implemented yet
        print("ERROR: EXP OVERFLOW");
        return [int(sgn)] + [1]*exp_len + [0]*man_len
    if(exp>0):
        exp = int2bin(exp,FP64_EXP_LEN)[FP64_EXP_LEN - exp_len:] 
        # Convert Mantissa
        man = strdnum(man64[0:man_len])
    else:
        # Denorm
        man = ([0]*abs(exp)+[1]+strdnum(man64))[0:man_len]
        exp = int2bin(0,exp_len)
    return [int(sgn)] + exp + man;



man = [0,0,0,0,0,0,0,0,0,0]
exp = [0,0,0,0,1]
pack = [0] + exp + man;
a = fpdecode(FMT_FP16,pack)
print(a)

man = [1,1,1,1,1,1,1,1,1,1]
exp = [0,0,0,0,0]
pack = [0] + exp + man;
a = fpdecode(FMT_FP16,pack)
print(a)

man = [0,0,0,0,0,0,0,0,0,1]
exp = [0,0,0,0,0]
pack = [0] + exp + man;
a = fpdecode(FMT_FP16,pack)
print(a)

man = [0,0,0,0,0,0,0,0,0,0]
exp = [0,0,0,0,0]
pack = [1] + exp + man;
a = fpdecode(FMT_FP16,pack)
print(a)

6.103515625e-05
6.097555160522461e-05
5.960464477539063e-08
-0.0


In [507]:
[0]*10

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [548]:
# FP16 Back-and-forth Conversion Test
# https://en.wikipedia.org/wiki/Half-precision_floating-point_format

def fp16verify(s, desc):
    t = fpdecode(FMT_FP16,strdnum(s));
    # print(t);
    k = numdstr(fpencode(FMT_FP16,t));
    if k==s:
        print('- pass -', desc)
    else:
        print('- FAIL - ', desc)
    print('Expected:\t', s);
    print('Result  :\t', k);

fp16verify('0000000000000001','Smallest Positive Subnormal')
fp16verify('0000001010101010','Subnormal')
fp16verify('0000001111111111','Largest Subnormal')
fp16verify('0000010000000000','Smallest Positive Normal')
fp16verify('0011010101010101','Nearest Value to 1/3')
fp16verify('0011101111111111','Largest Number Less Than One')
fp16verify('0011110000000000','One')
fp16verify('0011110000000001','Smallest Number Larger Than One')
fp16verify('0111101111111111','Largest Normal Number')
fp16verify('0011010101010101','Nearest Value to 1/3')
fp16verify('0111110000000000','Infinity')
fp16verify('1000000000000000','Negative Zero')
fp16verify('0000000000000000','Positive Zero')
fp16verify('1100000000000000','-2')
fp16verify('1111111111111111','-Infinity')

- pass - Smallest Positive Subnormal
Expected:	 0000000000000001
Result  :	 0000000000000001
- pass - Subnormal
Expected:	 0000001010101010
Result  :	 0000001010101010
- pass - Largest Subnormal
Expected:	 0000001111111111
Result  :	 0000001111111111
- pass - Smallest Positive Normal
Expected:	 0000010000000000
Result  :	 0000010000000000
- pass - Nearest Value to 1/3
Expected:	 0011010101010101
Result  :	 0011010101010101
- pass - Largest Number Less Than One
Expected:	 0011101111111111
Result  :	 0011101111111111
- pass - One
Expected:	 0011110000000000
Result  :	 0011110000000000
- pass - Smallest Number Larger Than One
Expected:	 0011110000000001
Result  :	 0011110000000001
- pass - Largest Normal Number
Expected:	 0111101111111111
Result  :	 0111101111111111
- pass - Nearest Value to 1/3
Expected:	 0011010101010101
Result  :	 0011010101010101
ERROR: SPECIAL NUMBER
- pass - Infinity
Expected:	 0111110000000000
Result  :	 0111110000000000
- pass - Negative Zero
Expected:	 1000000000

In [487]:
bin2int('0101010010101')
import math
def convexp(exp, exp_len, exp_bias):
    exp = bin2int(exp) - FP64_EXP_BIAS + exp_bias
    if (math.log(abs(exp))/math.log(2)) >= exp_len:
        # Overflow, not implemented yet
        print("ERROR: EXP OVERFLOW");
        return []
    exp = int2bin(exp,FP64_EXP_LEN)[FP64_EXP_LEN - exp_len:]
    return exp
    
convexp('01111110001',5,15)

[0, 0, 0, 0, 1]